In [6]:
import asyncpg

async def create_connection_pool():
    return await asyncpg.create_pool(
        user='your_user',
        password='your_password',
        database='your_database',
        host='127.0.0.1',
        port=5432,
        min_size=1,
        max_size=5,
    )

In [ ]:
class UserRepository:
    def init(self, db_pool):
        self.db_pool = db_pool

    async def create_table(self):
        async with self.db_pool.acquire() as conn:
            await conn.execute('''
                CREATE TABLE IF NOT EXISTS users (
                    id SERIAL PRIMARY KEY,
                    name TEXT NOT NULL,
                    age INT
                );
            ''')

    async def add_user(self, name: str, age: int):
        async with self.db_pool.acquire() as conn:
            await conn.execute(
                'INSERT INTO users (name, age) VALUES ($1, $2);',
                name, age
            )

    async def get_all_users(self):
        async with self.db_pool.acquire() as conn:
            rows = await conn.fetch('SELECT * FROM users;')
            return [dict(row) for row in rows]

    async def get_user_by_id(self, user_id: int):
        async with self.db_pool.acquire() as conn:
            row = await conn.fetchrow('SELECT * FROM users WHERE id = $1;', user_id)
            return dict(row) if row else None